In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# evaluate accuracy of the original networks before finetuning
from onnx2torch import convert
from stitchnet.stitchonnx.utils import accuracy_score_net,Net,get_score_net,load_cats_and_dogs_dset,load_dl,eval_original_model,get_macs_params_onnx,get_data_score
from stitchnet.stitchonnx.utils import accuracy_score_net_plants
from stitchnet.stitchonnx.utils import load_hf_train_val_dset
from stitchnet.stitchonnx.viz import draw_stitchNet_fromTuples
import torch
import os
from glob import glob
from tqdm import tqdm
from skl2onnx.helpers.onnx_helper import load_onnx_model
from pathlib import Path
data_score = get_data_score(32)

batch_size = 32
from stitchnet.stitchonnx.utils import load_hf_train_val_dset_with_test_split

dataset_train, dataset_val = load_hf_train_val_dset_with_test_split('food101',train='train', val='validation',label="label",num_train=1000, num_val=250)

dataloaders = dict(
    train=load_dl(dataset_train, batch_size, shuffle=True),
    val=load_dl(dataset_val, batch_size, shuffle=False)
)
RESULT_NAME = f"_results/original_food"
os.makedirs(RESULT_NAME, exist_ok=True)

modelnames = sorted(glob('_models/*.onnx'))
for i,modelname in tqdm(enumerate(modelnames)):
    name = os.path.basename(modelname)
    namewithoutext = os.path.splitext(name)[0]
    # if os.path.exists(f'_results/original/{namewithoutext}.txt'):
    #     continue
    model_onnx1 = load_onnx_model(modelname)
    macs, params = get_macs_params_onnx(model_onnx1)
    torch_model_1 = convert(model_onnx1)
    # valacc,trainacc = eval_original_model(torch_model_1, dataloaders)
    
    # get score
    fragmentFiles = sorted(glob(f'_models/fragments/net{i:03}/*.onnx'))
    onnxFragments = []
    js = []
    for j,fragmentFile in enumerate(fragmentFiles):
        onnxFragment = load_onnx_model(fragmentFile)
        onnxFragments.append(onnxFragment)
        js.append((i,j))
    net1 = Net(onnxFragments, i)
    score = get_score_net(net1, data_score)
    
    valacc = accuracy_score_net_plants(Net([model_onnx1]), dataset_val, dataset_train, bs=256)
    trainacc = accuracy_score_net_plants(Net([model_onnx1]), dataset_train, dataset_train, bs=256)
    print('VALACC:', valacc, 'TRAINACC:', trainacc)
    
    with open(f'{RESULT_NAME}/{namewithoutext}.txt', 'w') as f:
        f.write(f'{valacc},{trainacc},{macs},{params},{score},"{tuple(js)}"\n')
        
    draw_stitchNet_fromTuples(js, name=f'{RESULT_NAME}/{namewithoutext}')
        # f.write(f'{",".join([f"{x:.4}" for x in [valacc,trainacc,macs,params,score]])}\n')
    # break

Found cached dataset food101 (/home/jupyter-steerapi/.cache/huggingface/datasets/food101/default/0.0.0/7cebe41a80fb2da3f08fcbef769c8874073a86346f7fb96dc0847d4dfc318295)


  0%|          | 0/2 [00:00<?, ?it/s]

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
Loading cached shuffled indices for dataset at /home/jupyter-steerapi/.cache/huggingface/datasets/food101/default/0.0.0/7cebe41a80fb2da3f08fcbef769c8874073a86346f7fb96dc0847d4dfc318295/cache-463b34067ffe427e.arrow
Loading cached shuffled indices for dataset at /home/jupyter-steerapi/.cache/huggingface/datasets/food101/default/0.0.0/7cebe41a80fb2da3f08fcbef769c8874073a86346f7fb96dc0847d4dfc318295/cache-31728fa0f16d1941.arrow
Loading cached processed dataset at /home/jupyter-steerapi/.cache/huggingface/datasets/food101/default/0.0.0/7cebe41a80fb2da3f08fcbef769c8874073a86346f7fb96dc0847d4dfc318295/cache-72bf87a1369951f8.arrow
Loading cached processed dataset at /home/jupyter-steerapi/.cache/huggingface/datasets/food101/default/0.0.0/7cebe41a80fb2da3f08fcbef769c8874073a86346f7fb96dc0847d4dfc318295/cache-83c43638744cba52.

input.4 input.4 0.9999999403953552
input.12 input.12 1.0



3it [00:05,  1.22s/it]
4it [00:05,  1.27it/s]

...

(1000, 1000) (1000,)



  0%|          | 0/4 [00:00<?, ?it/s]2023-07-11 20:33:00.395089155 [W:onnxruntime:, execution_frame.cc:812 VerifyOutputSizes] Expected shape from model of {1,1000} does not match actual shape of {256,1000} for output 70

 25%|██▌       | 1/4 [00:01<00:04,  1.41s/it]2023-07-11 20:33:01.019088102 [W:onnxruntime:, execution_frame.cc:812 VerifyOutputSizes] Expected shape from model of {1,1000} does not match actual shape of {256,1000} for output 70

 50%|█████     | 2/4 [00:01<00:01,  1.08it/s]2023-07-11 20:33:01.581975237 [W:onnxruntime:, execution_frame.cc:812 VerifyOutputSizes] Expected shape from model of {1,1000} does not match actual shape of {256,1000} for output 70

 75%|███████▌  | 3/4 [00:02<00:00,  1.32it/s]2023-07-11 20:33:02.152990981 [W:onnxruntime:, execution_frame.cc:812 VerifyOutputSizes] Expected shape from model of {1,1000} does not match actual shape of {232,1000} for output 70

100%|██████████| 4/4 [00:03<00:00,  1.20it/s]
5it [02:02, 32.23s/it]

VALACC: 0.18 TRAINACC: 1.0


/home/jupyter-steerapi/.conda/envs/stitchnet/lib/python3.10/site-packages/onnx_tool/node_profilers.py:1127: UserWarning: node Reshape_99 cannot reshape array of size (197,1,2304,) into shape (197,12,64,) 
  warnings.warn(f'node {self.name} cannot reshape array of size {tuple2str(srcshape)} into shape {tuple2str(shape)} ')
/home/jupyter-steerapi/.conda/envs/stitchnet/lib/python3.10/site-packages/onnx_tool/node_profilers.py:1127: UserWarning: node Reshape_101 cannot reshape array of size (197,1,2304,) into shape (197,12,64,) 
  warnings.warn(f'node {self.name} cannot reshape array of size {tuple2str(srcshape)} into shape {tuple2str(shape)} ')
/home/jupyter-steerapi/.conda/envs/stitchnet/lib/python3.10/site-packages/onnx_tool/node_profilers.py:1127: UserWarning: node Reshape_102 cannot reshape array of size (197,1,2304,) into shape (197,12,64,) 
  warnings.warn(f'node {self.name} cannot reshape array of size {tuple2str(srcshape)} into shape {tuple2str(shape)} ')
/home/jupyter-steerapi/.co

InferenceError: [TypeInferenceError] Inferred elem type differs from existing elem type: (INT64) vs (FLOAT)

In [3]:
# from stitchnet.stitchonnx.utils import load_dl,load_onnx_model,load_cats_and_dogs_dset,execute_fragments,convert_imagenet_to_cat_dog_label
# import numpy as np
# from glob import glob
# from tqdm import tqdm

# dataset_val = load_cats_and_dogs_dset("test")
# fragmentFiles = sorted(glob(f'_models/fragments/net001/*.onnx'))
# onnxFragments = []
# js = []
# for j,fragmentFile in enumerate(fragmentFiles):
#     onnxFragment = load_onnx_model(fragmentFile)
#     onnxFragments.append(onnxFragment)

In [4]:
print("DONE")

DONE
